In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/Users/hirofumi48/162348.github.io/posts/2025/Comp"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
n, p, pₑ = 200, 50, 10

using Random, StatsFuns, Distributions
β_true = vcat(randn(pₑ), zeros(p - pₑ))
X = randn(n, p)

η_true = X * β_true
π_true = logistic.(η_true)

y = rand.(Bernoulli.(π_true))
y = collect(Float64, y)

200-element Vector{Float64}:
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 ⋮
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0

In [3]:
using PolyaGammaHybridSamplers, LinearAlgebra

function pg_logistic_gibbs(
    X::Matrix{Float64},
    y::Vector{Float64};
    n_iter::Int = 5000,
    burnin::Int = 1000,
    σ_prior::Float64 = 10.0,
)
    n, p = size(X)

    # 事前: β ~ N(0, σ_prior^2 I)
    V0_inv = (1 / σ_prior^2) * LinearAlgebra.I  # precision of prior

    # 初期値
    β = zeros(p)
    κ = y .- 0.5  # κ_i = y_i - 1/2

    # サンプル保存用
    n_save = n_iter - burnin
    β_samples = Matrix{Float64}(undef, n_save, p)

    # メインループ
    for it in 1:n_iter
        # 1. PG 補助変数 ω_i | β のサンプル
        η = X * β
        ω = similar(η)
        for i in 1:n
            pg = PolyaGammaHybridSampler(1.0, η[i])
            ω[i] = rand(pg)
        end

        # 2. β | ω, y のサンプル (多変量ガウス)
        Ω = Diagonal(ω)
        precision = X' * Ω * X + V0_inv          # posterior precision
        cov = inv(Matrix(precision))             # posterior covariance
        m = cov * (X' * κ)                       # posterior mean (μ0=0 のため)

        # β ~ N(m, cov)
        β = rand(MvNormal(m, Symmetric(cov)))

        # burn-in 後に保存
        if it > burnin
            β_samples[it - burnin, :] .= β
        end
    end

    return β_samples
end

σ_prior = 10.0
β_samples_pg = pg_logistic_gibbs(X, y;
    n_iter = 6000,
    burnin = 1000,
    σ_prior = σ_prior,
)

5000×50 Matrix{Float64}:
 3.8757   4.36705  -0.698822   9.02867  …   0.87445    -1.49918    0.200862
 4.40732  4.2971   -0.280925   9.01887      0.512335   -1.42127    0.595946
 5.21268  4.52158  -1.25717    8.68178      0.159297   -1.5986     0.616531
 4.7013   4.81096  -1.32211    8.3189      -0.249103   -1.38248    0.228233
 4.62719  4.2955   -1.11457    8.94093      0.206265   -0.736113   0.397653
 5.05957  4.52355  -0.98997    8.70049  …  -0.113928   -0.958431   0.120708
 4.65216  4.02931  -0.689566   8.82764     -0.319217   -0.7996     0.0607134
 4.90388  3.34935  -0.995185   8.7343       0.119835   -0.166864   0.205527
 3.76524  4.11204  -1.15605    8.20476     -0.0833992  -0.662408   0.25657
 3.99344  4.45968  -1.88764    7.94347      0.47089    -0.976165  -0.32334
 4.25664  4.26288  -1.98098    8.64629  …   0.402839   -0.899734   0.0819894
 4.51715  4.56934  -1.75408    8.19242      0.32849    -0.631458   0.202072
 3.77384  3.78269  -1.55313    7.74139      0.0381279  -1.01578

In [4]:
using MCMCChains
# Chains 型にして Turing の結果と揃える（オプション）
names = Symbol.("β[$i]" for i in 1:p)
chain_pg = Chains(β_samples_pg, names)

Chains MCMC chain (5000×50×1 Array{Float64, 3}):

Iterations        = 1:1:5000
Number of chains  = 1
Samples per chain = 5000
parameters        = β[1], β[2], β[3], β[4], β[5], β[6], β[7], β[8], β[9], β[10], β[11], β[12], β[13], β[14], β[15], β[16], β[17], β[18], β[19], β[20], β[21], β[22], β[23], β[24], β[25], β[26], β[27], β[28], β[29], β[30], β[31], β[32], β[33], β[34], β[35], β[36], β[37], β[38], β[39], β[40], β[41], β[42], β[43], β[44], β[45], β[46], β[47], β[48], β[49], β[50]

Use `describe(chains)` for summary statistics and quantiles.


In [5]:
using Turing, LinearAlgebra

@model function logreg_turing(x, y, σ_prior)
    n, p = size(x)
    
    # 事前分布
    β ~ MvNormal(zeros(p), (σ_prior^2) * I)
    
    # ベクトル化した尤度（高速化）
    η = x * β
    y ~ arraydist(Bernoulli.(logistic.(η)))
end

σ_prior = 10.0
model = logreg_turing(X, y, σ_prior)

n_samples = 200
n_adapt   = 100

chain_hmc = sample(
    model,
    NUTS(n_adapt, 0.6),
    n_samples,
)

┌ Info: Found initial step size
└   ϵ = 0.8


Sampling:   6%|██▌                                      |  ETA: 0:00:02

Sampling:  32%|█████████████▏                           |  ETA: 0:00:02

Sampling:  48%|███████████████████▋                     |  ETA: 0:00:01

Sampling:  57%|███████████████████████▎                 |  ETA: 0:00:01

...

Sampling:  80%|████████████████████████████████▋        |  ETA: 0:00:00

Sampling:  88%|███████████████████████████████████▉     |  ETA: 0:00:00

Sampling:  96%|███████████████████████████████████████▎ |  ETA: 0:00:00

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:01


Chains MCMC chain (200×64×1 Array{Float64, 3}):

Iterations        = 101:1:300
Number of chains  = 1
Samples per chain = 200
Wall duration     = 6.61 seconds
Compute duration  = 6.61 seconds
parameters        = β[1], β[2], β[3], β[4], β[5], β[6], β[7], β[8], β[9], β[10], β[11], β[12], β[13], β[14], β[15], β[16], β[17], β[18], β[19], β[20], β[21], β[22], β[23], β[24], β[25], β[26], β[27], β[28], β[29], β[30], β[31], β[32], β[33], β[34], β[35], β[36], β[37], β[38], β[39], β[40], β[41], β[42], β[43], β[44], β[45], β[46], β[47], β[48], β[49], β[50]
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Use `describe(chains)` for summary statistics and quantiles.


In [6]:
using MCMCChains, Statistics

summarize(chain_hmc)



  parameters      mean       std      mcse   ess_bulk   ess_tail      rhat   e ⋯
      Symbol   Float64   Float64   Float64    Float64    Float64   Float64     ⋯

        β[1]    4.3778    1.0104    0.1250    64.4417   100.4983    1.0238     ⋯
        β[2]    5.3844    1.0756    0.1138    90.1820    91.6947    1.0131     ⋯
        β[3]   -0.7699    0.6707    0.0526   168.7842   181.8165    1.0018     ⋯
        β[4]    8.6576    1.4172    0.1924    53.6985    52.3139    1.0535     ⋯
        β[5]    0.5906    0.7380    0.0556   173.5834   154.0692    1.0014     ⋯
        β[6]   -3.4345    0.8361    0.0877    99.5875    68.0840    1.0215     ⋯
        β[7]    3.6595    0.7915    0.0802    99.4618   165.9748    1.0161     ⋯
        β[8]    2.7890    0.8168    0.0776   105.4048   182.9626    0.9997     ⋯
        β[9]   -8.5910    1.3622    0.1871    51.9045    73.1560    1.0313     ⋯
       β[10]   -3.0966    0.7775    0.0788    98.0379   110.1591    1.0160     ⋯
       β[11]    0.6272   

In [7]:
using Statistics

# 真の β との誤差
mean_hmc = vec(mean(Array(chain_hmc), dims=1))  # ここは実際のパラメータ名に合わせて調整
mean_pg = vec(mean(Array(chain_pg), dims=1))

println("‖β̂_HMC - β_true‖₂ = ", norm(mean_hmc .- β_true))
println("‖β̂_PG  - β_true‖₂ = ", norm(mean_pg  .- β_true))

# ランタイムや ESS の比較も：
ess_hmc = ess_rhat(chain_hmc)
ess_pg  = ess_rhat(chain_pg)

‖β̂_HMC - β_true‖₂ = 12.265143635744415


‖β̂_PG  - β_true‖₂ = 12.6167543112654


ESS/R-hat

  parameters        ess      rhat   ess_per_sec 
      Symbol    Float64   Float64       Missing 

        β[1]    96.5919    1.0228       missing
        β[2]   105.6180    1.0095       missing
        β[3]   437.2263    1.0017       missing
        β[4]    84.8278    1.0183       missing
        β[5]   493.9429    1.0003       missing
        β[6]   158.6810    1.0116       missing
        β[7]   132.9422    1.0094       missing
        β[8]   147.4065    1.0173       missing
        β[9]    81.2850    1.0196       missing
       β[10]   130.5956    1.0106       missing
       β[11]   457.7798    1.0021       missing
       β[12]   490.0962    1.0013       missing
       β[13]   569.6477    0.9999       missing
       β[14]   282.2019    1.0056       missing
       β[15]   441.6704    1.0014       missing
       β[16]   299.3586    1.0038       missing
       β[17]   515.1474    1.0021       missing
       β[18]   161.3479    1.0075       missing
       β[19]   610.5524   